In [0]:
# %sql

#         select * from
#         (SELECT *
#                 FROM (
#                     SELECT 
#                     TRY_CAST(get_json_object(feedata, '$.estimatedtotaldue') AS DOUBLE) as invoiceamount, 
#                     TRY_CAST(get_json_object(feedata, '$.amountpaid') AS DOUBLE) as chargedamount, 
#                     TRY_CAST(get_json_object(feedata, '$.feewaiveramount') AS DOUBLE) as feewaived, 
#                     created_at,
#                     createdby,
#                     cfrfeeid,
#                     ministryrequestid,
#                     --*,
#                         ROW_NUMBER() OVER (
#                             PARTITION BY cf.ministryrequestid 
#                             ORDER BY cf.created_at DESC
#                         ) AS rn
#                     FROM foi_mod.foirequestcfrfees cf
#                     where created_at > '2025-08-01'
#                 ) sub
#                 WHERE rn = 1) sq

#         join 

#         (SELECT *
#                 FROM (
#                     SELECT 
#                     created_at,
#                     createdby,
#                     ministryrequestid,
#                     --*,
#                         ROW_NUMBER() OVER (
#                             PARTITION BY cf.ministryrequestid 
#                             ORDER BY cf.created_at DESC
#                         ) AS rn
#                     FROM foi_mod.foirequestcfrfees cf
#                     where TRY_CAST(cfrfeestatusid AS INT) = 2 --approved
#                 ) sub
#                 WHERE rn = 1) sq2 on sq.ministryrequestid = sq2.ministryrequestid

#         join

#         (SELECT *
#                 FROM (
#                     SELECT 
#                     created_at,
#                     createdby,
#                     ministryrequestid,
#                     --*,
#                         ROW_NUMBER() OVER (
#                             PARTITION BY cf.ministryrequestid 
#                             ORDER BY cf.created_at asc
#                         ) AS rn
#                     FROM foi_mod.foirequestcfrfees cf
#                 ) sub
#                 WHERE rn = 1) sq3 on sq3.ministryrequestid = sq2.ministryrequestid

#         join 
#         (select foirequest_id, foiministryrequestid from foi_mod.foiministryrequests) sq4 on sq4.foiministryrequestid = sq.ministryrequestid

In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
from pyspark.sql.functions import collect_list, concat_ws
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

runcycleid = etl_helpers.start_run_cycle("factRequestInvoices")

try:
    today = str(datetime.date.today())

    df_existing = spark.sql("SELECT max(TIMESTAMP(runcycleendat)) as runcycleendat from dimruncycle where packagename = 'factRequestInvoices' and success = 't'")
    df_existing.show()
    maxcreatedate = df_existing.first().runcycleendat
    print(maxcreatedate)
    maxcreatedate_str = maxcreatedate.strftime("%Y-%m-%d %H:%M:%S")

    df_existing = spark.sql(f"select distinct foirequest_id from foi_mod.foiministryrequests where foiministryrequestid in (SELECT distinct ministryrequestid from foi_mod.foirequestcfrfees where (created_at > '{maxcreatedate_str}' or TRY_CAST(updated_at AS DATE) > '{maxcreatedate_str}'))")

    comma_list_df = df_existing.select(concat_ws(",", collect_list("foirequest_id")).alias("comma_list"))
    comma_list = comma_list_df.collect()[0]["comma_list"]
    print(comma_list)

    query = f"""MERGE INTO default.factrequestinvoices dd
        -- USING dimRequests r -- commented out since we are adding sourceoftruth column to each fact table
        -- ON r.foirequestid = dd.foirequestid
        WHEN MATCHED and dd.foirequestid in ({comma_list}) and sourceoftruth = 'FOIMOD' THEN

        UPDATE 
        SET dd.activeflag = 'N'"""

    print(query)


    query = f"""
        select * from
        (SELECT *
                FROM (
                    SELECT 
                    TRY_CAST(get_json_object(feedata, '$.estimatedtotaldue') AS DOUBLE) as invoiceamount, 
                    TRY_CAST(get_json_object(feedata, '$.amountpaid') AS DOUBLE) as chargedamount, 
                    TRY_CAST(get_json_object(feedata, '$.feewaiveramount') AS DOUBLE) as feewaived, 
                    created_at,
                    createdby,
                    cfrfeeid,
                    ministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY cf.ministryrequestid 
                            ORDER BY cf.created_at DESC
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees cf
                    where created_at > '{maxcreatedate_str}'
                ) sub
                WHERE rn = 1) sq

        join 

        (SELECT *
                FROM (
                    SELECT 
                    created_at,
                    createdby,
                    ministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY cf.ministryrequestid 
                            ORDER BY cf.created_at DESC
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees cf
                    where TRY_CAST(cfrfeestatusid AS INT) = 2 --approved
                ) sub
                WHERE rn = 1) sq2 on sq.ministryrequestid = sq2.ministryrequestid

        join

        (SELECT *
                FROM (
                    SELECT 
                    created_at,
                    createdby,
                    ministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY cf.ministryrequestid 
                            ORDER BY cf.created_at asc
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees cf
                ) sub
                WHERE rn = 1) sq3 on sq3.ministryrequestid = sq2.ministryrequestid

        join 
        (select foirequest_id, foiministryrequestid from foi_mod.foiministryrequests) sq4 on sq4.foiministryrequestid = sq.ministryrequestid
        """

    print(query)

    df = spark.sql(query)
    print(df.count())
    df.show()

    if (df.count() == 0):
        raise Exception("no changes for today")

    # order of columns here is important!
    df_mapped = df.selectExpr(
        "foirequest_id as foirequestid",
        f"{runcycleid} as runcycleid",
        "cfrfeeid as invoiceid",
        "NULL as invoicenumber",
        "sq3.createdby AS createdby",
        "sq3.created_at AS createddate",
        "sq.createdby AS modifiedby",
        "sq.created_at AS modifieddate",
        "'' as comments",
        "'' AS correspondenceid",
        "invoiceamount AS invoiceamount",
        "chargedamount AS chargedamount",
        "'' as incurredamount",
        "feewaived AS feewaived",
        "'' as admincostpercentage",
        "'' as chargedadmincostamount",
        "NULL as incurredadmincostamount",
        "NULL as feewaivedadmincost",
        "'' as annotationtext",
        "sq2.created_at AS invoiceddate",
        "'' as approvalstatus",
        "sq2.createdby AS approvedby",
        "sq2.created_at AS approveddate",
        "'Y' as activeflag",
        "'FOIMOD' as sourceoftruth"
    )
    df_mapped.show()
    df_mapped.write.format("delta").mode("append").option("mergeSchema", "false").insertInto("factrequestinvoices")  
    etl_helpers.end_run_cycle(runcycleid, 't', "factRequestInvoices")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestInvoices", "Credentials not available")
    raise Exception("notebook failed") from e
except Exception as e:    
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "factRequestInvoices")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestInvoices", f"An error occurred: {e}")
        raise Exception("notebook failed") from e